# Actors manipulation

The core of **ABM** is the definition of individual behaviour of actors. However, when implementing a program, it is necessary to operate on individuals that satisfy certain conditions. The resulting diversity of actor behaviour is one of the biggest challenges in implementing SES modelling. To this end, `ABSESpy` provides a number of practical methods to manipulate actors.

## Selecting actors

First create some subjects for testing.

In [1]:
from abses import Actor, MainModel


class Farmer(Actor):
    pass


class Worker(Actor):
    pass


model = MainModel(name="test", base="tests")

# create 15 actors in sum.
farmers = model.agents.create(Farmer, 5)
workers = model.agents.create(Worker, 5)

# Randomly add them to the world.
model.nature.add_agents(model.agents.to_list())

There are three ways to create expression for selecting all farmers in the model. (Search by breed.)

In [2]:
selecting_farmers1 = "breed == Farmer"
selecting_farmers2 = "Farmer"
selecting_farmers3 = {"breed": "Farmer"}

model.agents.select(selecting_farmers1)
model.agents.select(selecting_farmers2)

<ActorsList: (5)Farmer>

<ActorsList: (5)Farmer>

For more complex cases, consider use the third way in selection.

In [3]:
# Label each actor for selecting
for i in range(5):
    farmers[i].label = i
    workers[i].label = i

# selecting the worker label = 3
model.agents.select({"breed": "Worker", "label": 3})

<ActorsList: (1)Worker>

`ActorsList.better()` method can select a subset of the actors whose `metric` is larger than a number...

In [4]:
farmers.better("label", 3)

<ActorsList: (1)Farmer>

## Define collections

The `Human` module can store the selection expression, a variable that is inert and will automatically return the actor that satisfies the condition any time it is needed in the future.

In [5]:
selected = model.human.define("farmer", {"breed": "Farmer", "label": 3})

model.human.farmer == selected
model.human.farmer

True

<ActorsList: (1)Farmer>

Only 1 eligible farmer was selected, but when the attribute was modified, the defined "Farmer" attribute returned the current number of eligible farmers (5).

In [6]:
farmers.label = 3

model.human.farmer == selected
model.human.farmer

False

<ActorsList: (5)Farmer>

## Actors' rules

Rules are conditional statements that persisted work for actors by default. 
User can define some rules to a group of actors.
Every time when those actors' attribute are changed, they will check all the rules that apply to themselves.
Once the conditions are met, run the next rules as triggered by the rules.

In [7]:
from abses.main import MainModel
from abses.actor import Actor

# create atesting actors
model = MainModel(name="testing rules", base="tests")


class TestActor(Actor):
    def report_breach(self):
        print(f"{self} broke regulation!")


# creating five testing actors, id from 5 to 10.
actors = model.agents.create(TestActor, 5)
actors.id

array([5, 6, 7, 8, 9])

In [8]:
import numpy as np

# define a rule for all actors.
actors.rule(when="test == 1", then="report_breach", disposable=True)

# Updating attributes
for index, actor in enumerate(actors):
    print(f"In looping step {index}:")
    actor.test = index

[None, None, None, None, None]

In looping step 0:
In looping step 1:
TestActor (Obj 6) broke regulation!
In looping step 2:
In looping step 3:
In looping step 4:
